# WAVE tutorial: a smart home

WAVE is platform for delegable authorization. We are going to test it out by looking at some use cases in a smart home context. You can also use WAVE for securing other things, like APIs, but this is a fun tutorial to cover the basics.

## Open up a second tab

We are going to open a new tab for this same notebook that you can leave pointing at your virtual devices by right clicking the notebook tab and clicking "New View for Notebook"

![New View](scaffold/NewView.png)

Note that due to a quirk of Jupyter Lab, one of your tabs will display the devices, while the other will say "Error creating widget". This is normal. Use the one without the devices for doing the exercises, and the one with the devices just for checking on the smart home state.

## Getting started

The lines below will "boot" your house. There is a home server that acts as the root of authority for your house and all its devices. Give your house a unique nickname, as later you will be accessing data from other people's houses. 

In [ ]:
from scaffold.tut import *

# This is a name that other people around you will use to communicate with you. It is also used
# to partition your communications on the shared MQTT server, so change it to something that nobody
# else will pick, but it still easy enough to type when you get to exercises that require you to
# share your nickname with someone else
my_unique_nickname="john-smith" # <<< CHANGE THIS

# wave is a connection to the WAVE daemon, that you will use for authorization actions
# homeserver is the control interface to your home server, allowing you to request permissions etc.
# mqtt is a connection to the rise camp MQTT server that routes all commands and sensor information 
wave, homeserver, mqtt = Initialize(my_unique_nickname)

## Your virtual home

The set of devices rendered above represent your virtual home for this tutorial. The lightbulb represents the lights in your house. The front door motion sensor is a way to see if someone is at your door. The thermostat both does control of your house temperature but also offers an **occupied** signal that tells you if there are people home.

The homeserver object has two useful functions. One is the `.namespace()` function which returns the hash of the namespace entity corresponding to your smart house. The other is for getting permissions, which we will see later. Try out the `namespace()` function now:

In [ ]:
print ("my namespace is: ",hashToBase64(homeserver.namespace()))

## Creating your entity

An *entity* represents a person or device. Your home server has an entity which is the root of permissions in the house, but every occupant of the house also needs their own entity so they can be granted permissions to interact with the house. Go ahead and create your entity by running the command below. 

Note that usually the command would be `entity = wave.CreateEntity(wv.CreateEntityParams())` but to make sure you don't lose your entity if you restart the notebook, we have made a wrapper that saves it to disk

In [ ]:
entity, _ = createOrLoadEntity(wave, "myEntity")

# a perspective is the form of an entity you use in calls to WAVE
perspective = wv.Perspective(entitySecret=wv.EntitySecret(DER=entity.SecretDER))

## Proofs of permission

Let's try control the house with this brand new entity. To do so, we need to form a *proof of permissions* (spoiler: it should fail):


In [ ]:
lightproof = wave.BuildRTreeProof(wv.BuildRTreeProofParams(
    # our secret entity perspective
    perspective=perspective,
    # the domain of authority we are accessing. In this case it is OUR homeserver
    namespace=homeserver.namespace(),
    # Tell WAVE to look for new permissions
    resyncFirst=True,
    # what we want to prove
    statements=[
        wv.RTreePolicyStatement(
            # This is a constant that represents a SET of permissions, so the permissions
            # strings can be disambiguated (otherwise "write" could apply to every application)
            permissionSet=smarthome_pset,
            permissions=["write"],
            # This is the control resource for the light. There is also a /report interface
            resource="smarthome/light/control",
        )
    ]
))

# lets check if that worked:
if lightproof.error.code != 0:
    print ("Proof building failed:", lightproof.error.message)
else:
    print ("Proof building succeeeded")

If this is the first time you've run the notebook, the above will fail. This is because this newly created entity has not been granted any permissions! Why should it be able to control the house? In real life, you would now communicate with a person/service that has the permissions you want and give them your entity hash. With this, they can grant you permissions. We will emulate that with a simple function call:

In [ ]:
homeserver.grant_permissions_to(entity.hash)

That grants you permissions to everything within the house, later we will do a permission grant from scratch, so you can see how it is done. For now, lets try building a proof again. This time it might take a little longer because it is busy discovering and decrypting all the new permissions that you just granted:

In [ ]:
lightproof = wave.BuildRTreeProof(wv.BuildRTreeProofParams(
    perspective=perspective,
    namespace=homeserver.namespace(),
    resyncFirst=True,
    statements=[
        wv.RTreePolicyStatement(
            permissionSet=smarthome_pset,
            permissions=["write"],
            resource="smarthome/light/control",
        )
    ]
))

if lightproof.error.code != 0:
    print ("Proof building failed:", lightproof.error.message)
else:
    print ("Proof building succeeeded")

This should succeed. We can now attach this proof to a command we send to the light, and it will obey the command because it can verify we are authorized. For this demo, the smart home devices rendered at the top of the notebook are listening for commands on an MQTT topic unique to your nickname.

In [ ]:
mqtt.publish(my_unique_nickname+"/smarthome/light/control",
#                      you can also use {"state":"off"}
             composeMessage(lightproof, {"state":"on"}))

If you scroll to the top of the notebook you should see that the light has turned on. To make it easier to see, you can add a copy of your smart home devices here:

In [ ]:
homeserver.render()

In addition to controlling lights, you can also form a proof for writing to the message box:

In [ ]:
msgproof = wave.BuildRTreeProof(wv.BuildRTreeProofParams(
    perspective=perspective,
    namespace=homeserver.namespace(),
    resyncFirst=True,
    statements=[
        wv.RTreePolicyStatement(
            permissionSet=smarthome_pset,
            permissions=["write"],
            # This is the resource for the message box
            resource="smarthome/notify",
        )
    ]
))

if msgproof.error.code != 0:
    print ("Proof building failed:", msgproof.error.message)
else:
    print ("Proof building succeeeded")


You send messages to it the same way you do for the light:

In [ ]:
mqtt.publish(my_unique_nickname+"/smarthome/notify",
                  composeMessage(msgproof, "hello world"))

Now that you have sent some authorization proofs attached to messages, let's walk through how you would decrypt and validate a message that you receive. Let's subscribe to the thermostat report topic. This will give us a stream of messages that we can process and use to make the light mirror the occupancy report from the thermostat.

NOTE: this tutorial is trying to teach you how to use WAVE at a lower level. Typically you would use something like WAVEMQ instead of MQTT which would do all the message decryption and validation for you, but for the purposes of teaching how to integrate WAVE with third party applications, we are doing it from scratch:

In [ ]:
def thermostat_cb(msg):
    # first decrypt the message
    resp = wave.DecryptMessage(wv.DecryptMessageParams(
        perspective= perspective,
        ciphertext= msg.payload,
        resyncFirst= True))
    if resp.error.code != 0:
        print ("dropping thermostat message:", resp.error.message)
        return
    # then break it up into proof + body
    proof, body = decomposeMessage(resp.content)
    
    # now validate the proof
    resp = wave.VerifyProof(wv.VerifyProofParams(
        proofDER=proof,
        requiredRTreePolicy=wv.RTreePolicy(
            namespace=homeserver.namespace(),
            statements=[wv.RTreePolicyStatement(
                permissionSet=smarthome_pset,
                permissions=["write"],
                resource="smarthome/thermostat/report",
            )]
        )
    ))
    if resp.error.code != 0:
        print ("dropping message: ", resp.error.message)
        return
        
    # the proof is valid!
    house_occupied = body["occupied"] # True or False
    
    print ("Received valid occupancy message. Occupied=", house_occupied)
    # fill in some code here to turn the light on if the house is occupied, and off otherwise 
    # HINT: you can copy from a previous cell and make a small change
    
    
mqtt.subscribe(my_unique_nickname+"/smarthome/thermostat/report", thermostat_cb)

If you get stuck, you can click the ellipsis below to expand the solution

In [ ]:
# SOLUTION:
def thermostat_cb(msg):
    # first decrypt the message
    resp = wave.DecryptMessage(wv.DecryptMessageParams(
        perspective= perspective,
        ciphertext= msg.payload,
        resyncFirst= True))
    if resp.error.code != 0:
        print ("dropping thermostat message:", resp.error.message)
        return
    # then break it up into proof + body
    proof, body = decomposeMessage(resp.content)
    
    # now validate the proof
    resp = wave.VerifyProof(wv.VerifyProofParams(
        proofDER=proof,
        requiredRTreePolicy=wv.RTreePolicy(
            namespace=homeserver.namespace(),
            statements=[wv.RTreePolicyStatement(
                permissionSet=smarthome_pset,
                permissions=["write"],
                resource="smarthome/thermostat/report",
            )]
        )
    ))
    if resp.error.code != 0:
        print ("dropping message: ", resp.error.message)
        return
        
    # the proof is valid!
    house_occupied = body["occupied"] # True or False
    
    print ("Received valid occupancy message. Occupied=", house_occupied)
    # SOLUTION:
    mqtt.publish(my_unique_nickname+"/smarthome/light/control",
        composeMessage(lightproof, {"state":"on" if house_occupied else "off"}))
    
    
mqtt.subscribe(my_unique_nickname+"/smarthome/thermostat/report", thermostat_cb)

You have now created a controller that consumes secure feeds and produces a control feed! The thermostat occupancy value changes by itself every few seconds, so you should be able to see your light turn on when the occupied value (to the right of the thermostat) reads true.

**Important** you must run the cell below to replace your controller with a no-op otherwise it will fight with later exercises:


In [ ]:
mqtt.subscribe(my_unique_nickname+"/smarthome/thermostat/report", lambda x:x)

## Interacting across namespaces

You are a good neighbor and you like to help out. Your neighbor is away on vacation, so you want to tie their house lights to yours so that their house seems occupied.

First off, you need to find a partner (like the person next to you!) and you need to communicate your Entity and Namespace hashes to them. Instead of typing them, let's use a shared MQTT topic. Fill in the same partnership nickname here, and then **both run the cell at the same time**

In [ ]:
# both you and your partner should pick the SAME unique partnership name here
our_partnership_nickname = "JohnSmithAndBobNeighborhood" # <<<< CHANGE THIS

shared_topic="partnerships/"+our_partnership_nickname

# print out messages from people looking for partners
def looking_for_partner(msg):
    # you can comment this out and re-run the cell to turn off these messages if they are annoying
    print (str(msg.payload,"utf8"))
    pass

mqtt.subscribe(shared_topic, looking_for_partner)
time.sleep(3)
mqtt.publish(shared_topic, 
             "my nickname="+my_unique_nickname+
             "\nentity hash="+hashToBase64(entity.hash)+ 
             "\nhome namespace="+hashToBase64(homeserver.namespace()))

Get your neighbor to run the above as well, and fill in the fields below. Remember to run the cell after you fill it in!



In [ ]:
partner_nickname = "paste the nickname here"
partner_entity_hash = hashFromBase64( "paste the entity hash here" )
partner_home_namespace = hashFromBase64 ( "paste the namespace here" )

Decide between you and your partner who is going on vacation (their lights will be controlled) and who is doing the controlling. Each of you follow ONE of the following two sections. When you are done, you can switch

## Partner 1: Having your lights controlled:

You need to grant the controlling entity (stored in `partner_entity_hash`) permission to control your lights:

In [ ]:
_ = wave.CreateAttestation(wv.CreateAttestationParams(
    perspective=perspective,
    # this is your PARTNER's entity hash
    subjectHash=partner_entity_hash,
    publish=True,
    policy=wv.Policy(rTreePolicy=wv.RTreePolicy(
        # and you are granting them permissions to YOUR house
        namespace=homeserver.namespace(),
        indirections=5,
        statements=[
            wv.RTreePolicyStatement(
                permissionSet=smarthome_pset,
                permissions=["write"],
                resource="smarthome/light/control",
            )]
    ))))


Note that what you have just done is *delegation*: you received some permissions from the home server entity and you are passing on a subset of those permissions on to your partner. Unfortunately, this is all you have to do for this exercise, lean over and give your partner some help as they build up logic for controlling your lights

## Partner 2: Controlling your partner's lights:

You need to build a proof that you can control your partner's light:

In [ ]:
partnerlightproof = wave.BuildRTreeProof(wv.BuildRTreeProofParams(
    perspective=perspective,
    # we want a proof we can control on THEIR namespace
    namespace=partner_home_namespace,
    # Tell WAVE to look for new permissions
    resyncFirst=True,
    # what we want to prove
    statements=[
        wv.RTreePolicyStatement(
            permissionSet=smarthome_pset,
            permissions=["write"],
            resource="smarthome/light/control",
        )
    ]
))

if partnerlightproof.error.code != 0:
    print ("Proof building failed:", partnerlightproof.error.message)
else:
    print ("Proof building succeeeded")

If your proof building failed, make sure your partner has run the granting cell above. Once it succeeds, you can use this proof to publish a message. This topic uses THEIR nickname, so it will be delivered to their homeserver that will validate the proof and act upon it.


In [ ]:
mqtt.publish(partner_nickname+"/smarthome/light/control", composeMessage(partnerlightproof, {"state":"on"}))

By filling in the gaps below, subscribe to your own light state and send commands that mirror that state onto your partner's light state.

In [ ]:
def light_controller_cb(msg):
    # the message we are getting here is our OWN light state
    # we need to first decrypt the message
    resp = wave.DecryptMessage(wv.DecryptMessageParams(
        perspective= perspective,
        ciphertext= msg.payload,
        resyncFirst= True))
    if resp.error.code != 0:
        print ("dropping light state message:", resp.error.message)
        return
    # then break it up into proof + body
    proof, body = decomposeMessage(resp.content)
    
    # now validate the proof
    resp = wave.VerifyProof(wv.VerifyProofParams(
        proofDER=proof,
        requiredRTreePolicy=wv.RTreePolicy(
            namespace= FILL_ME_IN, #<< fill this in, what namespace should it be?
            statements=[wv.RTreePolicyStatement(
                permissionSet=smarthome_pset,
                permissions=["write"],
                resource="smarthome/light/report",
            )]
        )
    ))
    if resp.error.code != 0:
        print ("dropping message: ", resp.error.message)
        return
        
    # the proof is valid!
    light_state = body["state"] # "on" or "off"
    
    # fill in some code here to send a command to your partner's light that mirrors
    # your own light state.   
    
mqtt.subscribe(my_unique_nickname+"/smarthome/light/report", light_controller_cb)

If you get stuck, you can click the ellipsis below to expand the solution

In [ ]:
def light_controller_cb(msg):
    # the message we are getting here is our OWN light state
    # we need to first decrypt the message
    resp = wave.DecryptMessage(wv.DecryptMessageParams(
        perspective= perspective,
        ciphertext= msg.payload,
        resyncFirst= True))
    if resp.error.code != 0:
        print ("dropping light state message:", resp.error.message)
        return
    # then break it up into proof + body
    proof, body = decomposeMessage(resp.content)
    
    # now validate the proof
    resp = wave.VerifyProof(wv.VerifyProofParams(
        proofDER=proof,
        requiredRTreePolicy=wv.RTreePolicy(
            namespace= homeserver.namespace(), #SOLUTION: our namespace
            statements=[wv.RTreePolicyStatement(
                permissionSet=smarthome_pset,
                permissions=["write"],
                resource="smarthome/light/report",
            )]
        )
    ))
    if resp.error.code != 0:
        print ("dropping message: ", resp.error.message)
        return
        
    # the proof is valid!
    light_state = body["state"] # "on" or "off"
    # SOLUTION:
    mqtt.publish(partner_nickname+"/smarthome/light/control", 
         composeMessage(partnerlightproof, {"state":light_state})) 
    
mqtt.subscribe(my_unique_nickname+"/smarthome/light/report", light_controller_cb)

You have now bound your light to your partner's light. Go turn your light on and off and make sure your partner's light also changes.

## Bonus section: decrypting your partner's streams

If you have a little extra time, feel free to try this one: Your vacation-happy neighbor is having a problem with package theft. He has asked you to create a controller that tells you when he gets a package and he is not home. Everyone in the neighborhood has a smart home with a motion sensor by the front door. They also have thermostats that have occupancy sensors. If there is motion in front of the neighbor's house, and nobody is home, then maybe you should go check for a package.

Discuss with your partner who is going on vacation, and who is the kind neighbor picking up packages:

## Partner 1: Going on vacation

You will need to grant your partner the ability to decrypt your thermostat data (which tells them if the house is occupied by the house sitter or not) and the ability to decrypt the motion sensor data (which tells them a package was delivered.

In [ ]:
# grant your partner the ability to decrypt your thermostat data
_ = wave.CreateAttestation(wv.CreateAttestationParams(
    perspective=perspective,
    subjectHash=partner_entity_hash,
    publish=True,
    policy=wv.Policy(rTreePolicy=wv.RTreePolicy(
        namespace=homeserver.namespace(),
        indirections=5,
        statements=[
            wv.RTreePolicyStatement(
                # This is a permission set used for special permissions
                permissionSet=wv.WaveBuiltinPSET,
                # this special permission generates end-to-end decryption keys
                permissions=[wv.WaveBuiltinE2EE],
                resource="smarthome/thermostat/report",
            )]
    ))))

# and grant them the ability to decrypt your motion sensor
_ = wave.CreateAttestation(wv.CreateAttestationParams(
    perspective=perspective,
    subjectHash=partner_entity_hash,
    publish=True,
    policy=wv.Policy(rTreePolicy=wv.RTreePolicy(
        namespace=homeserver.namespace(),
        indirections=5,
        statements=[
            wv.RTreePolicyStatement(
                permissionSet=wv.WaveBuiltinPSET,
                permissions=[wv.WaveBuiltinE2EE],
                resource="smarthome/motion/report",
            )]
    ))))

## Partner 2: The kind neighbor

To check you have permissions, lets try building a proof that you can decrypt the motion sensor data:


In [ ]:
decryptmotionproof = wave.BuildRTreeProof(wv.BuildRTreeProofParams(
    perspective=perspective,
    # the namespace will be our partner's home, not ours
    namespace=partner_home_namespace,
    # Tell WAVE to look for new permissions
    resyncFirst=True,
    # what we want to prove
    statements=[
        wv.RTreePolicyStatement(
            permissionSet=wv.WaveBuiltinPSET,
            permissions=[wv.WaveBuiltinE2EE],
            resource="smarthome/motion/report",
        )
    ]
))

if decryptmotionproof.error.code != 0:
    print ("Proof building failed:", decryptmotionproof.error.message)
else:
    print ("Proof building succeeeded")

If the above build failed, check your partner has run the granting cell.

Now, by consulting the subscription callbacks you have written in the past, write a controller that listens to and decrypts your neighbors motion and occupancy data, and publishes a notification message to your own homeserver if you need to check for a package because the house sitter is not home. The rough structure looks like this:


In [ ]:
partner_occupied = False

def partner_motion_cb(msg):
    global partner_occupied

    # Fill in:
    
    # Decrypt message
    # Verify message
    # If partner_occupied is false, write message to your own notification box
    # Don't worry about the body of the message, the existence of a message implies
    # there was a package delivered
        
        
# This one is done for you
def partner_occupancy_cb(msg):
    global partner_occupied
    # we need to first decrypt the message
    resp = wave.DecryptMessage(wv.DecryptMessageParams(
        perspective= perspective,
        ciphertext= msg.payload,
        resyncFirst= True))
    if resp.error.code != 0:
        print ("dropping occupancy message:", resp.error.message)
        return
    # then break it up into proof + body
    proof, body = decomposeMessage(resp.content)
    
    # now validate the proof
    resp = wave.VerifyProof(wv.VerifyProofParams(
        proofDER=proof,
        requiredRTreePolicy=wv.RTreePolicy(
            # this is from our partner's namespace
            namespace=partner_home_namespace,
            statements=[wv.RTreePolicyStatement(
                permissionSet=smarthome_pset,
                permissions=["write"],
                resource="smarthome/thermostat/report",
            )]
        )
    ))
    if resp.error.code != 0:
        print ("dropping occupancy message: ", resp.error.message)
        return
    
    partner_occupied = body["occupied"]
    
mqtt.subscribe(partner_nickname+"/smarthome/thermostat/report", partner_occupancy_cb)
mqtt.subscribe(partner_nickname+"/smarthome/motion/report", partner_motion_cb)

If you get stuck, you can click the ellipsis below to expand the solution

In [ ]:
partner_occupied = False

def partner_motion_cb(msg):
    global partner_occupied
    # we need to first decrypt the message
    resp = wave.DecryptMessage(wv.DecryptMessageParams(
        perspective= perspective,
        ciphertext= msg.payload,
        resyncFirst= True))
    if resp.error.code != 0:
        print ("dropping motion message:", resp.error.message)
        return
    # then break it up into proof + body
    proof, body = decomposeMessage(resp.content)
    
    # now validate the proof
    resp = wave.VerifyProof(wv.VerifyProofParams(
        proofDER=proof,
        requiredRTreePolicy=wv.RTreePolicy(
            # this is from our partner's namespace
            namespace=partner_home_namespace,
            statements=[wv.RTreePolicyStatement(
                permissionSet=smarthome_pset,
                permissions=["write"],
                resource="smarthome/motion/report",
            )]
        )
    ))
    if resp.error.code != 0:
        print ("dropping motion message: ", resp.error.message)
        return
    
    if !partner_occupied:
        mqtt.publish(my_unique_nickname+"/smarthome/notify",
             composeMessage(msgproof, "a package was delivered to your partner!"))
        
# This one is done for you
def partner_occupancy_cb(msg):
    global partner_occupied
    # the message we are getting here is our OWN light state
    # we need to first decrypt the message
    resp = wave.DecryptMessage(wv.DecryptMessageParams(
        perspective= perspective,
        ciphertext= msg.payload,
        resyncFirst= True))
    if resp.error.code != 0:
        print ("dropping motion message:", resp.error.message)
        return
    # then break it up into proof + body
    proof, body = decomposeMessage(resp.content)
    
    # now validate the proof
    resp = wave.VerifyProof(wv.VerifyProofParams(
        proofDER=proof,
        requiredRTreePolicy=wv.RTreePolicy(
            namespace=partner_home_namespace,
            statements=[wv.RTreePolicyStatement(
                permissionSet=smarthome_pset,
                permissions=["write"],
                resource="smarthome/thermostat/report",
            )]
        )
    ))
    if resp.error.code != 0:
        print ("dropping message: ", resp.error.message)
        return
    
    partner_occupied = body["occupied"]
    
mqtt.subscribe(partner_nickname+"/smarthome/thermostat/report", partner_occupancy_cb)
mqtt.subscribe(partner_nickname+"/smarthome/motion/report", partner_motion_cb)